In [ ]:
%load_ext autoreload
%autoreload 2
from scipy import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# from dag_prf_utils.plot_functions import *
%matplotlib inline

import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [8]:
dm = io.loadmat('GazeCenterFS_vd.mat')['stim']

# braincoder convention is time by x by y
paradigm = np.rollaxis(dm, 2, 0)
aspect_ratio = paradigm.shape[2] / paradigm.shape[1]

x, y = np.meshgrid(np.linspace(-.75, .75, 3), np.linspace(-aspect_ratio *.75, aspect_ratio *.75, 3))

In [9]:
parameters = pd.DataFrame({'x':x.ravel(),
               'y':y.ravel(),
               'sd':.2,
               'amplitude':1,
               'baseline':0.0}).astype(np.float32)

y_grid, x_grid = np.meshgrid(np.linspace(-aspect_ratio, aspect_ratio, paradigm.shape[2]), 
                              np.linspace(-1., 1., paradigm.shape[1]))

grid_coordinates = np.stack((x_grid.ravel().astype(np.float32), y_grid.ravel().astype(np.float32)), 1)

y_grid, x_grid = np.meshgrid(np.linspace(-aspect_ratio, aspect_ratio, paradigm.shape[2]), 
                              np.linspace(-1., 1., paradigm.shape[1]))

grid_coordinates = np.stack((x_grid.ravel().astype(np.float32), y_grid.ravel().astype(np.float32)), 1)

In [10]:
from braincoder.models import GaussianPRF2DWithHRF
from braincoder.hrf import SPMHRFModel

In [11]:
model = GaussianPRF2DWithHRF(grid_coordinates, 
                      paradigm=paradigm,
                     parameters=parameters,
                      hrf_model=SPMHRFModel(tr=1.7))

In [ ]:
for i in np.arange(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(model.get_rf()[i].reshape((240, 135)).T)
    plt.axis("off")

## Predictions (no noise)

In [ ]:
pred = model.predict()
pred.plot(legend=False)

## Noisy predictions

In [ ]:
data = model.simulate(noise=2)
data.plot(legend=False)
sns.despine()

# Estimate parameters

In [ ]:
from braincoder.optimize import ParameterFitter
from braincoder.bprf_mcmc import BPRF

In [ ]:
model = GaussianPRF2DWithHRF(grid_coordinates, 
                      paradigm=paradigm,
                     parameters=parameters,
                      hrf_model=SPMHRFModel(tr=1.7))
cfitter = ParameterFitter(model, data, paradigm)


grid_pars = cfitter.fit_grid(x=np.linspace(-1, 1, 20), 
                            y=np.linspace(-aspect_ratio, aspect_ratio, 20),
                            sd=np.linspace(0.01, 1., 10),
                            baseline=[0.0],
                            amplitude=[1.0],
                            use_correlation_cost=True)
ols_pars = cfitter.refine_baseline_and_amplitude(grid_pars)
refined_pars = cfitter.fit(init_pars=ols_pars)

In [ ]:
cfitter.get_rsq(refined_pars)

In [ ]:
idx = 0
from braincoder.utils.visualize import quick_plot
quick_plot(
    model=model, 
    parameters=refined_pars.iloc[idx,:],
    data=data.iloc[:,idx]
)

In [ ]:
240*135

# MCMC Gaussian

In [19]:
bounds = {
    'x' : [-1, 1],
    'y' : [-1, 1],
    'sd' : [0, 10],
    'baseline' : [-1,1],
    'amplitude' : [0, 10], 
}

n_vx = parameters.shape[0]
init_pars = pd.DataFrame({    
    'x' : np.zeros(n_vx),
    'y' : np.zeros(n_vx),
    'sd' : np.ones(n_vx),
    'amplitude' : np.ones(n_vx),
    'baseline' : np.zeros(n_vx),
})


bfitter = BPRF(model=model, data=data, )
# # bfitter.add_prior(
# #     pid='x', prior_type='normal', loc=0, scale=5
# # )
# # bfitter.add_prior(
# #     pid='y', prior_type='normal', loc=0, scale=5
# # )
# # bfitter.add_prior(
# #     pid='sd', prior_type='uniform', vmin=0, vmax=10
# # )
# # bfitter.add_prior(
# #     pid='amplitude', prior_type='uniform', vmin=0, vmax=100
# # )

# # bfitter.add_prior(
# #     pid='baseline', prior_type='uniform', vmin=-1, vmax=1
# # )
# bfitter.add_bijector(
#     'x', 'sigmoid', low=-1, high=1
# )
# bfitter.add_bijector(
#     'y', 'sigmoid', low=-1, high=1
# )
bfitter.prep_for_fitting()


In [ ]:
idx = 0
num_results = 50
tf.random.set_seed(1)
bfitter.fit(
    idx=idx,
    init_pars=init_pars,
    num_results=num_results,
)
# tf.random.set_seed(1)
# bfitterL.fit(
#     idx=idx,
#     init_pars=init_pars,
#     num_results=num_results,         
# )

In [ ]:
# Lets compare...
p = 'amplitude'
# plt.plot(
#     bfitterL.mcmc_sampler[idx][p],
#     color='r', label='latent versions')
plt.plot(
    bfitter.mcmc_sampler[idx][p],
    color='g',) #label='no bijector versions')
plt.axhline(
    parameters[p][idx], color='k'
)
plt.legend()

In [ ]:
rsq = bfitter.get_rsq_for_idx(
    idx=idx, parameters=bfitter.mcmc_sampler[idx],
)

rsq_cols = dag_get_col_vals(
    rsq, cmap='RdBu_r', vmin=-1, vmax=1,
    )
rsq

In [ ]:
idx = 0
burn_in = 0
sns_plot = sns.pairplot(
    bfitter.mcmc_sampler[idx].iloc[burn_in:,:],
    plot_kws={'color':rsq_cols[burn_in:,:]},
    # corner=True, 
    diag_kind='kde'
)
sns_plot.map_lower(sns.kdeplot, levels=4, color=0.2)
from dag_prf_utils.plot_functions import *
# sns_plot.
edit_pair_plot(
    sns_plot.axes, lines_dict=parameters.iloc[idx,:].to_dict(), color='g', linestyle=':', lw=4, label='Truth'
)
edit_pair_plot(
    sns_plot.axes, lim_dict=bounds,
)
# edit_pair_plot(
#     sns_plot.axes, lines_dict=init_pars.iloc[idx,:].to_dict(), color='c', linestyle=':', lw=4,
# )
# edit_pair_plot(
#     sns_plot.axes, lines_dict=refined_pars.iloc[idx,:].to_dict(), color='r', linestyle=':', lw=4,
# )
plt.legend()
plt.show()

In [ ]:
p = 'sd'
plt.plot(bfitter.mcmc_sampler[idx][p])
plt.axhline(parameters.iloc[idx][p], c='r')

# Mexican hat model

In [27]:
from braincoder.models import DifferenceOfGaussiansPRF2DWithHRF

In [ ]:
parameters_dog = pd.DataFrame({'x':x.ravel(),
               'y':y.ravel(),
               'sd':.2,
               'amplitude':0.3,
               'baseline':0.0,
               'srf_amplitude':.4, 
               'srf_size':2}).astype(np.float32)
model_dog = DifferenceOfGaussiansPRF2DWithHRF(grid_coordinates, paradigm, parameters=parameters_dog,
                                             hrf_model=SPMHRFModel(tr=1.8))
pred_dog = model_dog.predict()
pred_dog.plot()

In [ ]:
data_dog = model_dog.simulate(noise=.1)
data_dog.plot(legend=False)
sns.despine()

In [33]:
fitter_dog = ParameterFitter(model_dog, data_dog, paradigm)

In [34]:
init_pars_dog = refined_pars.copy()
init_pars_dog['srf_amplitude'] = 0.1
init_pars_dog['srf_size'] = 2.

In [ ]:
est_pars_dog = fitter_dog.fit(
    init_pars=init_pars_dog, learning_rate=0.001,
    max_n_iterations=5000,
    )

In [ ]:
idx = 0
est_pred = model_dog.predict(parameters=est_pars_dog)
est_pred[idx].plot()
data_dog[idx].plot()
print(est_pars_dog.iloc[idx,:])
print(parameters_dog.iloc[idx,:])


# MCMC - DOG

In [69]:
bounds_dog = {
    'x' : [-5, 5],
    'y' : [-5, 5],
    'sd' : [0, 10],
    'baseline' : [-2,2],
    'amplitude' : [0,100], 
    'srf_amplitude': [0, 10],
    'srf_size' : [1, 10],

}

n_vx = parameters_dog.shape[0]
init_pars_dog = pd.DataFrame({    
    'x' : np.zeros(n_vx),
    'y' : np.zeros(n_vx),
    'sd' : np.zeros(n_vx)+1,
    'baseline' : np.zeros(n_vx),
    'amplitude' : np.zeros(n_vx) + 1,
    'srf_amplitude' : np.zeros(n_vx) + .5,
    'srf_size' : np.zeros(n_vx) + 2,

})
bfitter_dog = BPRF(model=model_dog, data=data_dog, )
# for p in ['sd', 'amplitude', 'srf_amplitude', 'srf_size']:
#     bfitter_dog.add_bijector(
#         pid=p, bijector_type='sigmoid', vmin=bounds_dog[p][0], vmax=bounds_dog[p][1]
        
#     )
bfitter_dog.add_priors_from_bounds(bounds_dog, ) #prior_type='latent_uniform')
bfitter_dog.prep_for_fitting()

In [ ]:
bfitter_dog.model.parameter_labels


In [ ]:
bfitter_dog.fit(
    idx=[0,], #2,3], 
    init_pars=init_pars_dog,
    num_results=500,         
)

In [ ]:
idx = 0
burn_in = 0

rsq = bfitter_dog.get_rsq_for_idx(
    idx=idx, parameters=bfitter_dog.mcmc_sampler[idx],
)
plt.plot(rsq)
plt.figure()
rsq_cols = dag_get_col_vals(
    rsq, cmap='RdBu_r', vmin=-1, vmax=1,
    )

sns_plot = sns.pairplot(
    bfitter_dog.mcmc_sampler[idx].iloc[burn_in:,:],
    plot_kws={'color':rsq_cols[burn_in:,:]},
    # corner=True, 
    diag_kind='kde'
)
sns_plot.map_lower(sns.kdeplot, levels=4, color=0.2)
# sns_plot.
edit_pair_plot(
    sns_plot.axes, lines_dict=parameters_dog.iloc[idx,:].to_dict(), color='g', linestyle=':'
)
# edit_pair_plot(
#     sns_plot.axes, init_pars.iloc[idx,:].to_dict(), color='c', linestyle=':', lw=4,
# )
# edit_pair_plot(
#     sns_plot.axes, refined_pars.iloc[idx,:].to_dict(), color='r', linestyle=':', lw=4,
# )
plt.show()

In [ ]:
plt.plot(
    bfitter_dog.mcmc_sampler[idx].x
)

In [ ]:
bfitter_dog.mcmc_sampler[idx].shape

In [ ]:
bfitter_dog.mcmc_sampler[0].x